In [1]:
import numpy as np
import pandas as pd
import datetime as datetime
import os, sys, glob, re
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.width', 2200)
pd.options.mode.chained_assignment = None
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 800)
pd.set_option("display.max_colwidth", 100)

In [2]:
path = {'filepath': r'P:\Projects'}
os.chdir(path['filepath'])

In [3]:
df = pd.read_excel('WIP Censo do r_futebol - 11_2022.xlsx')

In [4]:
# Essa é parte mais antiga do script, mas no fim eu nem usei o dicionário porque eu simplesmente não sei trabalhar com dicionários no Python.
# O Coritiba só entrou pro Athletico ter um rival, mas a realidade é que bem poucos Coritibanos responderam o censo, e portanto o clube entrou no grupo dos "Outros" nos gráficos finais.


estados = list(('Rio Grande do Sul', 'Rio Grande do Sul', 'Paraná', 'Paraná', 'Minas Gerais', 'Minas Gerais', 'Rio de Janeiro', 'Rio de Janeiro', 'Rio de Janeiro', 'Rio de Janeiro', 'São Paulo', 'São Paulo', 'São Paulo', 'São Paulo'))
grandes = list(('Grêmio', 'Inter', 'Athletico-PR', 'Coritiba', 'Atlético-MG', 'Cruzeiro', 'Botafogo-RJ', 'Flamengo', 'Fluminense', 'Vasco', 'Corinthians', 'Palmeiras', 'Santos', 'São Paulo'))
grandesdict = dict(zip(grandes, estados))
grandesdict

{'Grêmio': 'Rio Grande do Sul',
 'Inter': 'Rio Grande do Sul',
 'Athletico-PR': 'Paraná',
 'Coritiba': 'Paraná',
 'Atlético-MG': 'Minas Gerais',
 'Cruzeiro': 'Minas Gerais',
 'Botafogo-RJ': 'Rio de Janeiro',
 'Flamengo': 'Rio de Janeiro',
 'Fluminense': 'Rio de Janeiro',
 'Vasco': 'Rio de Janeiro',
 'Corinthians': 'São Paulo',
 'Palmeiras': 'São Paulo',
 'Santos': 'São Paulo',
 'São Paulo': 'São Paulo'}

In [5]:
# No início a ideia era criar dummy variables pra cada membro do clube dos 14, mas no fim essa ideia não foi usada.


for i in grandes:
    df[i] = ((df.iloc[:, 43] == i) | (df.iloc[:, 46] == i))

In [6]:
# Esse script limpa a variável do clube principal.


def Clube(x):
    try:
        if x == 'Galo':
            return 'Atlético-MG'
        else:
            return x
    except Exception as e:
        return 'error'
    
    
df['Clube'] = df['Pra qual clube você torce? (Em caso de haver dois, aquele geograficamente mais próximo de você)'].apply(lambda row: Clube(row))
df['Clube'].value_counts()

Palmeiras                                                93
Flamengo                                                 91
Corinthians                                              87
São Paulo                                                66
Grêmio                                                   44
Fluminense                                               37
Atlético-MG                                              34
Inter                                                    33
Vasco                                                    31
Cruzeiro                                                 28
Santos                                                   21
Nenhum                                                   20
Botafogo-RJ                                              17
Athletico-PR                                             15
Bahia                                                    11
Ceará                                                    10
Coritiba                                

In [7]:
# Esse script define se a pessoa é associada com o estado do seu clube ou não (seja por nascer lá ou por morar lá)
# Perceba que o censo não tem como saber se a pessoa cresceu no estado e se mudou recentemente.


def Interestadual(x, y, z):
    try:
        for i in range(14):
            if z == grandes[i]:
                if x == estados[i] or y == estados[i]:
                    return False
                else:
                    return True
    except Exception as e:
        return 'error'

    
df['Interestadual'] = df.apply(lambda row: Interestadual(row['Onde você nasceu?'], 
                                                         row['Onde você mora atualmente?'], 
                                                         row['Clube']), axis = 1)
df['Interestadual'].value_counts()

False    455
True     150
Name: Interestadual, dtype: int64

In [8]:
# Esse script define três tipos de usuário, os não-usuários que vieram só pela Copa, os que têm menos de 2 anos de sub, e os que têm mais de 2 anos de sub.
# Perceba que muitos dos não-usuários vieram pela Copa mas continuarão participando do sub.
# O censo não tem como saber quais não-usuários continuarão participando e quais não.
# Os não-usuários foram excluídos de muitos dos gráficos finais, pois o intuito é fazer um censo do sub em si. Eles podem fácilmente ser reacrescentados se quisermos ver os gráficos com eles incluídos.


def Usuário(x):
    try:
        if x == 'Há 1 mês ou menos':
            return 'Não-usuário'
        elif x == 'Há menos de 6 meses, mas já acompanhava desde antes da Copa':
            return 'Usuário novo'
        elif x == 'Entre 6 meses e 1 ano':
            return 'Usuário novo'
        elif x == 'Entre 1 e 2 anos':
            return 'Usuário novo'
        elif x == 'Quase 2 anos (23 meses)':
            return 'Usuário novo'
        elif x == 'Entre 2 e 3 anos':
            return 'Usuário veterano'
        elif x == 'Entre 3 e 4 anos':
            return 'Usuário veterano'
        elif x == 'Entre 4 e 5 anos':
            return 'Usuário veterano'
        elif x == 'Mais de 5 anos':
            return 'Usuário veterano'
    except Exception as e:
        return 'error'
    
    
df['Usuário'] = df['Há quanto tempo você acompanha o /r/futebol?'].apply(lambda row: Usuário(row))
df['Usuário'].value_counts()

Usuário novo        413
Usuário veterano    278
Não-usuário          46
Name: Usuário, dtype: int64

In [9]:
# Existem duas maneiras de diferenciar os usuários: o jeito simples de 0 a 3, e um segundo método que dá um peso maior pra diferenciar ainda mais usuários que escrevem ou postam de lurkers.
# Essa classificação numérica acabou não sendo usada nos gráficos finais, apenas a classificação categórica original.


def TipoDeUsuárioSimples(x):
    try:
        if x == 'Leio as threads, mas raramente faço a minha própria ou leio os comentários':
            return 0
        elif x == 'Leio as threads e comentários, mas raramente escrevo o meu próprio':
            return 1
        elif x == 'Leio threads e comentários, nunca faço nenhum.':
            return 1
        elif x == 'Escrevo comentários, mas raramente crio threads':
            return 2
        elif x == 'Crio threads e escrevo comentários seguidamente':
            return 3
    except Exception as e:
        return 'error'
    
df['TipoDeUsuárioSimples'] = df['Que tipo de usuário você é?'].apply(lambda row: TipoDeUsuárioSimples(row))
display(df['TipoDeUsuárioSimples'].value_counts())


def TipoDeUsuárioWeighted(x):
    try:
        if x == 'Leio as threads, mas raramente faço a minha própria ou leio os comentários':
            return 0
        elif x == 'Leio as threads e comentários, mas raramente escrevo o meu próprio':
            return 1
        elif x == 'Leio threads e comentários, nunca faço nenhum.':
            return 1
        elif x == 'Escrevo comentários, mas raramente crio threads':
            return 3
        elif x == 'Crio threads e escrevo comentários seguidamente':
            return 4
    except Exception as e:
        return 'error'
    
    
df['TipoDeUsuárioWeighted'] = df['Que tipo de usuário você é?'].apply(lambda row: TipoDeUsuárioWeighted(row))
display(df['TipoDeUsuárioWeighted'].value_counts())

1.0    339
2.0    286
0.0     58
3.0     46
Name: TipoDeUsuárioSimples, dtype: int64

1.0    339
3.0    286
0.0     58
4.0     46
Name: TipoDeUsuárioWeighted, dtype: int64

In [10]:
df.columns

Index(['Timestamp', 'Onde você nasceu?', 'Se você não nasceu no Brasil, escreva o nome do país onde você nasceu, ou em Português ou em Inglês.', 'Onde você mora atualmente?', 'Se você não mora no Brasil, escreva o nome do país onde você mora, ou em Português ou em Inglês.', 'Com qual gênero você se identifica?', 'Qual é sua orientação sexual?', 'Qual é sua cor ou raça?', 'Qual é sua idade? Escreva em números aqui ou selecione sua faixa etária abaixo.', 'Se preferir, selecione sua faixa etária.', 'Qual é o seu grau de escolaridade?', 'Se você cursou ou está cursando o Ensino Superior, qual é sua área de formação?', 'Qual é sua situação no mercado de trabalho?', 'Qual é seu status de relacionamento?', 'Há quanto tempo você usa o reddit?', 'Numa escala de 1 (nem um pouco) a 10 (muito), o quanto você diria que é apaixonado pelo futebol? Escreva em números.', 'Há quanto tempo você acompanha o /r/futebol?', 'Que tipo de usuário você é?', 'Como você descobriu o /r/futebol?', 'Você costuma nav

In [11]:
# Esse script corrige usuários que escolheram o seu clube como o clube que ele simpatiza na resposta de "fora seus clubes de coração, com qual clube você mais simpatiza?"
# O Santos foi considerado como não ter rival, e o censo corrobora essa decisão, visto que a esmagadora maioria dos Santistas escolheram o Flamengo, enquanto que apenas um o Palmeiras, apenas um o Corinthians, um o Bragantino, e nenhum o São Paulo.


def Problema(x, y, z):
    try:
        if x == z:
            return True
        elif y == z:
            return True
        else:
            return False
    except Exception as e:
        return False
    
    
df['Problema'] = df.apply(lambda row: Problema(row['Clube'], 
                                               row['Qual é o segundo clube (aquele que fica geograficamente mais longe de você) pro qual você torce?'], 
                                               row['Fora o(s) seu(s) clube(s) do coração, com qual clube você mais simpatiza?']), axis = 1)
display(df['Problema'].value_counts())


def Solução(x, y):
    if y == True:
        return 'Problema'
    else:
        return x
    
    
df['Simpáticos'] = df.apply(lambda row: Solução(row['Fora o(s) seu(s) clube(s) do coração, com qual clube você mais simpatiza?'], 
                                                row['Problema']), axis = 1)
#display(df['Simpáticos'].value_counts())

False    761
True      14
Name: Problema, dtype: int64

In [12]:
# Esse script corrige usuários que escolheram um rival seu na resposta de "fora seus rivais, qual clube você mais quer ver perder?"
# O Santos foi considerado como não ter rival, e o censo corrobora essa decisão, visto que a esmagadora maioria dos Santistas escolheram o Flamengo, enquanto que apenas um o Palmeiras, apenas um o Corinthians, um o Bragantino, e nenhum o São Paulo.


def Problema2(x, y, z, s):
    try:
        if s == True:
            return False
        elif grandesdict[x] == grandesdict[z]:
            return True
        elif grandesdict[y] == grandesdict[z]:
            return True
        else:
            return False
    except Exception as e:
        return False
    
    
df['Problema2'] = df.apply(lambda row: Problema2(row['Clube'], 
                                                 row['Qual é o segundo clube (aquele que fica geograficamente mais longe de você) pro qual você torce?'], 
                                                 row['Fora o(s) rival(is), qual clube você mais quer ver perder?'], 
                                                 row['Santos']), axis = 1)
display(df['Problema2'].value_counts())


def Solução2(x, y, z):
    if y == True and z == False:
        return 'Problema'
    else:
        return x
    
    
df['Antipáticos'] = df.apply(lambda row: Solução2(row['Fora o(s) rival(is), qual clube você mais quer ver perder?'], 
                                                  row['Problema2'],
                                                  row['Santos']), axis = 1)

False    736
True      39
Name: Problema2, dtype: int64

In [13]:
# Essa foi a primeiras grande ideia que eu tive pro censo. Eu queria ver se o nível de sucesso do clube do usuário afeta o seu amor pelo futebol, a quantidade de partidas que ele assiste, etc.
# É uma hipótese um tanto plausível, mas não fará parte dos gráficos finais, pois o censo concluiu que não. O Flamenguista e o Palmeirense não amam o futebol mais que o Vascaíno nem assiste mais partidas que o Gremista só por estarem mais felizes com seus times.
# Por um lado até foi bom não ter dado certo, pois seria complicado eu ter que explicar como classifiquei cada time.
# Alguns são bem óbvio, ninguém nega que Flamengo, Palmeiras, e Athletico pertencem ao primeiro escalão. Ninguém também pode negar que Vasco, Cruzeiro, e Grêmio pertencem ao último.
# Já outros clubes são bem mais polêmicos.

def TipoDeTorcedor(a, b, c, d, e, f, g, h, i, j, k, l, m, n):
    if c == True or h == True or l == True:
        return 'torcedor_contente'
    elif e == True or i == True or k == True or m == True or n == True:
        return 'torcedor_normal'
    elif a == True or b == True or d == True or f == True or g == True or j == True:
        return 'torcedor_descontente'

    
df['Tipo de Torcedor'] = df.apply(lambda row: TipoDeTorcedor(row['Grêmio'], row['Inter'], row['Athletico-PR'], row['Coritiba'], row['Atlético-MG'], row['Cruzeiro'], row['Botafogo-RJ'], 
                                                             row['Flamengo'], row['Fluminense'], row['Vasco'], row['Corinthians'], row['Palmeiras'], row['Santos'], row['São Paulo']), axis = 1)

In [14]:
# Esse script limpa os dados de idade (e.g. cortar os "anos" de "17 anos") e tira a média pro pessoal que não quis colocar a idade exata.


def trim_age(x):
    return str(x)[:2]


df['Idade Limpa'] = df['Qual é sua idade? Escreva em números aqui ou selecione sua faixa etária abaixo.'].apply(lambda row: trim_age(row))


def Idade(x, y):
    try: 
        if int(x) > 0:
            return int(x)
    except Exception as e:
        if y == 'Menos de 16':
            return 14.5
        elif y == 'Entre 16 e 20':
            return 18
        elif y == 'Entre 21 e 24':
            return 22.5
        elif y == 'Entre 25 e 29':
            return 27
        elif y == 'Entre 30 e 34':
            return 32
        elif y == 'Entre 35 e 39':
            return 37
        elif y == 'Entre 40 e 44':
            return 42
        elif y == 'Mais de 44':
            return 46
    
    
df['Idade'] = df.apply(lambda row: Idade(row['Idade Limpa'],
                                         row['Se preferir, selecione sua faixa etária.']), axis = 1)
df['Idade'].value_counts()

27.0    121
18.0    100
22.5     74
32.0     63
23.0     33
25.0     32
26.0     30
24.0     27
20.0     24
29.0     23
22.0     23
28.0     19
17.0     18
14.5     17
21.0     16
19.0     15
30.0     14
16.0     14
37.0     14
31.0     12
42.0      8
33.0      7
34.0      6
36.0      5
14.0      4
39.0      3
35.0      2
50.0      1
40.0      1
90.0      1
15.0      1
44.0      1
99.0      1
45.0      1
46.0      1
12.0      1
38.0      1
Name: Idade, dtype: int64

In [15]:
# Esse script limpa os dados da escala de paixão.

def Paixão(x):
    try:
        if x <= 10:
            return x
        elif x > 10:
            return 10
    except Exception as e:
        if str(x).strip() == '7,5':
            return 7.5
        elif str(x).strip() == 'Oito.':
            return 8.8
        elif str(x).strip() == '8,8':
            return 8.8
        elif str(x).strip() == '8,8':
            return 8.67
        elif str(x).strip() == 'é deiz':
            return 10
        elif str(x).strip() == '8,5':
            return 8.5
        elif str(x).strip() == '9. 10 é só pelo meu time':
            return 9
        elif str(x).strip() == '9,86666...':
            return 9.8667
        elif str(x).strip() == '10 né malandro ta tirano':
            return 10
        elif str(x).strip() == '8 (sccp 7 x 1 santos)':
            return 8
    
    
df['Paixão'] = df['Numa escala de 1 (nem um pouco) a 10 (muito), o quanto você diria que é apaixonado pelo futebol? Escreva em números.'].apply(lambda row: Paixão(row))

In [16]:
# Esse script converte os dados de partidas assistidas por semana numa quantidade numérica.


def Assistir(x):
    try:
        if x == 'Nunca ou quase nunca assisto':
            return 0
        elif x == '1 ou 2 partidas por mês':
            return 0.3448
        elif x == '1 ou 2 partidas por semana':
            return 1.5
        elif x == '3 ou 4 partidas por semana':
            return 3.5
        elif x == '5 ou 6 partidas por semana':
            return 5.5
        elif x == '7 ou mais':
            return 7.5
        elif x == 'Uma':
            return 1
        elif x == '2 ou 3 por semana':
            return 2.5
    except Exception as e:
        return 'error'
    
    
df['Assistir'] = df['Quantas partidas de futebol você costuma assistir por semana?'].apply(lambda row: Assistir(row))

In [17]:
# Esse script converte os dados de partidas assistidas no estádio numa quantidade numérica.


def Estádio(x):
    try:
        if str(x) == 0 or x == 0:
            return 0
        elif x == '1 ou 2':
            return 1.5
        elif x == '3 ou 4':
            return 3.5
        elif x == '5 ou 6':
            return 5.5
        elif x == '7 ou 8':
            return 7.5
        elif x == '9 ou mais':
            return 9.5
        elif x == 'Todas do Palmeiras em casa menos duas, dá mais de 30':
            return 9.5
    except Exception as e:
        return 'error'

    
df['Estádio'] = df['Você assistiu a quantas partidas no estádio em 2022?'].apply(lambda row: Estádio(row))

In [18]:
# Esse script cria uma classificação numérica pro grau no qual o usuário assiste partidas como espectador.
# Essa classificação acabou não sendo usada nos gráficos finais.


def Espectador(x):
    try:
        if x == 'Não':
            return 0
        elif x == 'Apenas clássicos, decisões, ou entre times fortes':
            return 1
        elif x == 'Sim, mesmo sem ser um jogo importante':
            return 2
        elif x == 'Só vejo jogos importantes':
            return 1
    except Exception as e:
        return 'error'
    
    
df['Espectador'] = df['Você costuma assistir partidas sem ser nem seu time nem seu rival jogando?'].apply(lambda row: Espectador(row))

In [19]:
# Esse script tentou diferenciar usuários entre aqueles que moram longe e aqueles que moram perto do seu time.
# Essa classificação não deu certo e eu acabei desistindo dela.


def Longe(a, b, x, y):
    try:
        if a == 'Outro país' and b == 'Outro país':
            return False
        elif x == 'Nenhum':
            return 'Sem time'
        elif y == 'Não há' or y != y:
            if a == grandesdict[x] or b == grandesdict[x]:
                return False
            else:
                return True
        elif a == grandesdict[x] or a == grandesdict[y] or b == grandesdict[x] or b == grandesdict[y]:
            return False
        else:
            return True
    except Exception as e:
        return 'error'
    
    
df['Longe'] = df.apply(lambda row: Longe(row['Onde você nasceu?'],
                                         row['Onde você mora atualmente?'], 
                                         row['Clube'], 
                                         row['Qual é o segundo clube (aquele que fica geograficamente mais longe de você) pro qual você torce?']), axis = 1)

In [20]:
# Esse script define se o usuário torce pro clube dos 14 ou não.
# Essa classificação acabou não sendo usada nos gráficos finais.


def TorcedorDoClubeDos14(a, b, c, d, e, f, g, h, i, j, k, l, m, n):
    if a == True or b == True or c == True or d == True or e == True or f == True or g == True or h == True or i == True or j == True or k == True or l == True or m == True or n == True:
        return True
    else:
        return False

df['Torcedor do Clube dos 14'] = df.apply(lambda row: TorcedorDoClubeDos14(row['Grêmio'], row['Inter'], row['Athletico-PR'], row['Coritiba'], row['Atlético-MG'], row['Cruzeiro'], row['Botafogo-RJ'], 
                                                                           row['Flamengo'], row['Fluminense'], row['Vasco'], row['Corinthians'], row['Palmeiras'], row['Santos'], row['São Paulo']), axis = 1)

In [21]:
# Esse é o script mais importante de todos, pois ele é uma peça essencial da grande maioria dos gráficos finais.
# Ele define 4 grupos de torcedores.
# O primeiro grupo torce pra um dos 13 membros do clube dos 13, e não torce pra nenhum segundo clube Brasileiro.
# O segundo grupo torce pra 2 clubes Brasileiros.
# O terceiro grupo torce pra apenas um clube Brasileiro que não faz parte do clube dos 13.
# O quarto grupo não torce pra nenhum clube.
# A denominação "Confusos" foi criada pois muitos usuários responderam que não torcem pra um segundo clube, mas escolheram um segundo clube na lista, ou vice versa.


ClubesEstrangeiros = list(['River Plate', 'Boca Juniors', 'Bayern Munchen', 'Notingham forest', 'Liverpool', 'Montreal FC', 'Tottenham?', 'man. united', 'Porto', 'FC Porto', 'independiente de avellaneda, defensores de belgrano', 'Everton', 'Everton '])


def TorcedorÚnico(x, y, z):
    if x != 'Não há' and x == x and x != y and x != 'num torço não po' and z == 'Sim' and x != 'Não há. A ideia de que o 1º clube seja o mais próximo contradiz as pesquisas sobre o assunto, na maioria das vezes é o contrário.' and x != 'Torço pelo desenvolvimento dos clubes paraenses pra que eles voltem a figurar na primeira divisão novamente, apenas' and x not in ClubesEstrangeiros and y not in ClubesEstrangeiros:
        return '2 clubes'
    elif x == y:
        return x
    elif x != 'Não há' and x == x and z != 'Sim' and x not in ClubesEstrangeiros and y not in ClubesEstrangeiros:
        return 'Confusos'
    elif x == 'Não há' and z == 'Sim':
        return 'Confusos'
    elif x != x and z == 'Sim':
        return 'Confusos'
    elif z == 'Sim' and x in ClubesEstrangeiros:
        return 'Confusos'
    elif z == 'Sim' and y in ClubesEstrangeiros:
        return 'Confusos'
    elif y in grandes and y != 'Coritiba' and z != 'Sim':
        return y
    elif y == 'Porto' or y == 'FC Porto' or y == 'independiente de avellaneda, defensores de belgrano' or y == 'Everton' or y == 'Everton ' or x == 'River Plate':
        return x
    elif x == 'Tottenham Hotspur' or x == 'Manchester United' or x == 'Real Madrid':
        return y
    elif y != 'Nenhum' and y != 'Marte FC' and y != 'Merda rala fc':
        return 'Outros'
    elif y == 'Nenhum':
        return 'Nenhum'
    
    
df['Torcedor Único'] = df.apply(lambda row: TorcedorÚnico(row['Qual é o segundo clube (aquele que fica geograficamente mais longe de você) pro qual você torce?'], 
                                                          row['Clube'], 
                                                          row['Você se considera torcedor de dois clubes brasileiros?']), axis = 1)
df['Torcedor Único'].value_counts()

2 clubes                 96
Outros                   94
Palmeiras                78
Flamengo                 74
Corinthians              70
São Paulo                53
Confusos                 50
Grêmio                   37
Inter                    34
Fluminense               33
Atlético-MG              32
Vasco                    28
Cruzeiro                 25
Nenhum                   20
Santos                   18
Botafogo-RJ              15
Athletico-PR             14
Solte a minha família     1
Goran Pandev              1
Name: Torcedor Único, dtype: int64

In [22]:
# Esse script consiste de uma classificação numérica que indica se o usuário participa ou não do subreddit do seu clube.
# A classificação numérica acabou não sendo usada nos gráficos finais, mas sim a classificação categórica original.


def Subreddit(x):
    try:
        if x == 'Não':
            return 0
        elif x == 'Queria, mas não é ativo':
            return 0
        elif x == 'Sim, mas raramento crio threads ou escrevo comentários':
            return 1
        elif x == 'Sim, e crio threads e/ou escrevo comentários seguidamente':
            return 1
    except Exception as e:
        return 'error'
    
    
df['Subreddit'] = df['Você frequenta o subreddit do seu clube?'].apply(lambda row: Subreddit(row))
df['Subreddit'].value_counts()

0.0    365
1.0    362
Name: Subreddit, dtype: int64

In [23]:
# Esse script determina quanto tempo de r/futebol o usuário tem, em média.


def IdadeNoSub(x):
    try:
        if x == 'Há 1 mês ou menos':
            return 0.0417
        elif x == 'Há menos de 6 meses, mas já acompanhava desde antes da Copa':
            return 0.2917
        elif x == 'Entre 6 meses e 1 ano':
            return .75
        elif x == 'Entre 1 e 2 anos':
            return 1.5
        elif x == 'Quase 2 anos (23 meses)':
            return 1.9583
        elif x == 'Entre 2 e 3 anos':
            return 2.5
        elif x == 'Entre 3 e 4 anos':
            return 3.5
        elif x == 'Entre 4 e 5 anos':
            return 4.5
        elif x == 'Mais de 5 anos':
            return 5.5
    except Exception as e:
        return 'error'
    
    
df['Idade No Sub'] = df['Há quanto tempo você acompanha o /r/futebol?'].apply(lambda row: IdadeNoSub(row))
df['Idade No Sub'].value_counts()

1.5000    214
0.7500    133
2.5000    122
3.5000     83
0.2917     65
0.0417     46
5.5000     39
4.5000     34
1.9583      1
Name: Idade No Sub, dtype: int64

In [24]:
# Esse script serve pra ver quantos usuários torcem pra dois clubes rivais.
# Apesar de vermos vários desses exemplos como flair duplas no sub, dos 738 participantes do censo só houve 1 desses casos, portanto não fará parte dos gráficos finais.


def Cursed(x, y):
    try:
        if x == 'Grêmio' and y == 'Inter':
            return True
        elif x == 'Inter' and y == 'Grêmio':
            return True
        elif x == 'Cruzeiro' and y == 'Atlético-MG':
            return True
        elif x == 'Atlético-MG' and y == 'Cruzeiro':
            return True
        elif x == 'Palmeiras' and y == 'Corinthians':
            return True
        elif x == 'Corinthians' and y == 'Palmeiras':
            return True
        elif x == 'Vasco' and y == 'Flamengo':
            return True
        elif x == 'Flamengo' and y == 'Vasco':
            return True
        elif x == 'Fluminense' and y == 'Flamengo':
            return True
        elif x == 'Flamengo' and y == 'Fluminense':
            return True
        else:
            return False
    except Exception as e:
        return False
    
    
df['Cursed'] = df.apply(lambda row: Cursed(row['Clube'], 
                                           row['Qual é o segundo clube (aquele que fica geograficamente mais longe de você) pro qual você torce?']), axis = 1)

In [25]:
# Esse script exclui as respostas troll e as respostas em branco.


df['Exclusion'] = False

mask = df[(df['Se você não nasceu no Brasil, escreva o nome do país onde você nasceu, ou em Português ou em Inglês.']) == (df['Com qual gênero você se identifica?'])].index
df.loc[mask, 'Exclusion'] = True

mask = df[df['Se você não nasceu no Brasil, escreva o nome do país onde você nasceu, ou em Português ou em Inglês.'] == 'Pensa'].index
df.loc[mask, 'Exclusion'] = True

mask = df[df['Se você não nasceu no Brasil, escreva o nome do país onde você nasceu, ou em Português ou em Inglês.'] == 'Uzbequistão'].index
df.loc[mask, 'Exclusion'] = True

mask = df[df['Se você não nasceu no Brasil, escreva o nome do país onde você nasceu, ou em Português ou em Inglês.'] == 'Marte'].index
df.loc[mask, 'Exclusion'] = True

mask = df[df['Se você não nasceu no Brasil, escreva o nome do país onde você nasceu, ou em Português ou em Inglês.'] == 'Solte a minha família'].index
df.loc[mask, 'Exclusion'] = True

mask = df[df['Onde você mora atualmente?'] == 'safada'].index
df.loc[mask, 'Exclusion'] = True

mask = df[(df['Onde você nasceu?'].isna()) & (df['Com qual gênero você se identifica?'].isna())].index
df.loc[mask, 'Exclusion'] = True

df2 = df[df['Exclusion'] == False]

In [26]:
# Esse script só serve pra testar o desvio padrão do Tableau.


def Estádiooo(x):
    try:
        if str(x) == 0:
            return 0
        elif x == 0:
            return 0
        elif x == '1 ou 2':
            return 1.5
        elif x == '3 ou 4':
            return 3.5
        elif x == '5 ou 6':
            return 5.5
        elif x == '7 ou 8':
            return 7.5
        elif x == '9 ou mais':
            return 9.5
        elif x == 'Todas do Palmeiras em casa menos duas, dá mais de 30':
            return 9.5
        else:
            return None
    except Exception as e:
        return None

    
dftemp = df.copy()
dftemp['Estádiooo'] = df['Você assistiu a quantas partidas no estádio em 2022?'].apply(lambda row: Estádiooo(row))
dftemp['Estádiooo'].dropna().std()

2.4853297791457423

In [27]:
# Esse script cria várias tabelas pra tornar mais fácil os diversos cálculos abaixo que servem pra calcular as médias usadas nas linhas de referência e no texto do PowerPoint.

dfcalc1 = df2[(df2['Há quanto tempo você acompanha o /r/futebol?'] != 'Há 1 mês ou menos') & (df2['Torcedor Único'] != 'Confusos')].copy()
dfcalc2 = df2[(df2['Há quanto tempo você acompanha o /r/futebol?'] != 'Há 1 mês ou menos') & (df2['Torcedor Único'] != 'Confusos') & (df2['Torcedor Único'] != '2 clubes') & (df2['Torcedor Único'] != 'Nenhum')].copy()
dfcalc3 = df2[(df2['Há quanto tempo você acompanha o /r/futebol?'] != 'Há 1 mês ou menos') & (df2['Torcedor Único'] != 'Confusos') & (df2['Torcedor Único'] != '2 clubes') & (df2['Torcedor Único'] != 'Nenhum') & (df2['Torcedor Único'] != 'Outros')].copy()

In [28]:
dfcalc1['Paixão'].mean()

7.951173708920186

In [29]:
len(df2[df2['Há quanto tempo você acompanha o /r/futebol?'] == 'Há 1 mês ou menos'])/len(df2)

0.0625

In [30]:
dfcalc1['Assistir'].mean()

2.4927698587127156

In [31]:
dfcalc1['Estádio'].mean()

1.4068857589984352

In [32]:
dfcalc3['Interestadual'].mean()

0.22708333333333333

In [33]:
dfcalc3['Subreddit'].mean()

0.5954356846473029

In [49]:
len(dfcalc1[dfcalc1['Você se considera torcedor de dois clubes brasileiros?'] == 'Sim'])/len(dfcalc1)

0.13819875776397517

In [34]:
dfcalc2temp = dfcalc2[(dfcalc2['Você se considera torcedor de dois clubes brasileiros?'] == 'Não, e nem tenho opinião sobre esse assunto') | (df2['Você se considera torcedor de dois clubes brasileiros?'] == 'Não, mas respeito quem seja') | (df2['Você se considera torcedor de dois clubes brasileiros?'] == 'Não, e acho isso um absurdo')].copy()

display(len(dfcalc2temp[dfcalc2temp['Você se considera torcedor de dois clubes brasileiros?'] == 'Não, mas respeito quem seja']))
display(len(dfcalc2temp[dfcalc2temp['Você se considera torcedor de dois clubes brasileiros?'] == 'Não, mas respeito quem seja'])/len(dfcalc2temp))

c:\datascience\python377\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


269

0.5037453183520599

In [35]:
display(len(dfcalc2temp[dfcalc2temp['Você se considera torcedor de dois clubes brasileiros?'] == 'Não, e acho isso um absurdo']))
display(len(dfcalc2temp[dfcalc2temp['Você se considera torcedor de dois clubes brasileiros?'] == 'Não, e acho isso um absurdo'])/len(dfcalc2temp))

123

0.2303370786516854

In [36]:
display(len(dfcalc1[dfcalc1['Qual é sua cor ou raça?'] == 'Branco']))
display(len(dfcalc1[dfcalc1['Qual é sua cor ou raça?'] == 'Branco'])/len(dfcalc1))

440

0.6832298136645962

In [37]:
display(len(df2[df2['Há quanto tempo você acompanha o /r/futebol?'] == 'Mais de 5 anos']))
display(len(df2[df2['Há quanto tempo você acompanha o /r/futebol?'] == 'Mais de 5 anos'])/len(df2))

38

0.051630434782608696

In [38]:
display(len(dfcalc1[((dfcalc1['Como você acha que devem mudar os estaduais? (Tente selecionar a opção mais próxima da sua ideia)'] == 'Clubes grandes não devem participar, apenas os menores') | (df2['Como você acha que devem mudar os estaduais? (Tente selecionar a opção mais próxima da sua ideia)'] == 'Estadual tem que acabar') | (df2['Como você acha que devem mudar os estaduais? (Tente selecionar a opção mais próxima da sua ideia)'] == 'Clubes grandes devem entrar direto nas quartas ou nas semi finais'))]))
display(len(dfcalc1[((dfcalc1['Como você acha que devem mudar os estaduais? (Tente selecionar a opção mais próxima da sua ideia)'] == 'Clubes grandes não devem participar, apenas os menores') | (df2['Como você acha que devem mudar os estaduais? (Tente selecionar a opção mais próxima da sua ideia)'] == 'Estadual tem que acabar') | (df2['Como você acha que devem mudar os estaduais? (Tente selecionar a opção mais próxima da sua ideia)'] == 'Clubes grandes devem entrar direto nas quartas ou nas semi finais'))])/len(dfcalc1))

c:\datascience\python377\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


322

c:\datascience\python377\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


0.5

In [39]:
# Essa parte final do código serve pra conferir os dados limpos com relação às perguntas sobre times simpáticos e antipáticos.

#len(df2[df2['Simpáticos'] != 'Problema']['Simpáticos'].dropna())
df2[df2['Simpáticos'] != 'Problema']['Simpáticos'].dropna().value_counts().head(90).sum()

707

In [40]:
len(df2[df2['Antipáticos'] != 'Problema']['Antipáticos'].dropna())
#df2[df2['Antipáticos'] != 'Problema']['Antipáticos'].dropna().value_counts().head(90).sum()

686

In [41]:
df2[df2['Simpáticos'] != 'Problema']['Simpáticos'].value_counts()

Nenhum, torço pro meu clube e os outros que se danem                212
Vasco                                                                70
Fortaleza                                                            44
América-MG                                                           33
Corinthians                                                          26
Palmeiras                                                            24
Santos                                                               23
Cruzeiro                                                             21
Portuguesa                                                           19
Bahia                                                                19
Chape                                                                16
Botafogo-RJ                                                          13
Inter                                                                13
São Paulo                                                       

In [42]:
df2['Antipáticos'].value_counts()

Flamengo                                                            275
Apenas quero o mal pro(s) meu(s) rival(is)                          133
Palmeiras                                                            52
Corinthians                                                          50
Não quero o mal pra ninguém                                          42
Problema                                                             38
Atlético-MG                                                          31
Athletico-PR                                                         21
Inter                                                                14
Cruzeiro                                                              8
São Paulo                                                             8
Grêmio                                                                7
Vasco                                                                 5
Sport                                                           

In [43]:
# Esse script calcula quais clubes são mais populares entre usuários que torcem pra dois clubes.

df2clubes = df2[(df2['Você se considera torcedor de dois clubes brasileiros?'] == 'Sim') & (df2['Torcedor Único'] != 'Confusos')][['Clube', 'Qual é o segundo clube (aquele que fica geograficamente mais longe de você) pro qual você torce?']].copy()
primeiroclube = df2clubes['Clube'].value_counts().to_frame().reset_index()
segundoclube = df2clubes['Qual é o segundo clube (aquele que fica geograficamente mais longe de você) pro qual você torce?'].value_counts().to_frame().reset_index()
df2clubestotal = primeiroclube.merge(segundoclube, how='outer', on='index')
df2clubestotal = df2clubestotal.fillna(value=0)
df2clubestotal['Total2Clubes'] = df2clubestotal['Clube'] + df2clubestotal['Qual é o segundo clube (aquele que fica geograficamente mais longe de você) pro qual você torce?']
df2clubestotal.sort_values(by = 'Total2Clubes', ascending = False)

,index,Clube,Qual é o segundo clube (aquele que fica geograficamente mais longe de você) pro qual você torce?,Total2Clubes
1,Flamengo,11.0,13.0,24.0
0,Corinthians,11.0,6.0,17.0
2,Palmeiras,7.0,7.0,14.0
4,São Paulo,5.0,9.0,14.0
7,Vasco,3.0,6.0,9.0
3,Grêmio,6.0,2.0,8.0
42,Santos,1.0,6.0,7.0
20,Fluminense,1.0,4.0,5.0
5,Sampaio Corrêa,4.0,0.0,4.0
6,Bahia,3.0,1.0,4.0


In [44]:
df2 = df2.merge(df2clubestotal, how='left', left_on='Clube', right_on='index')
df2 = df2.drop(columns=['index', 'Clube_y', 'Qual é o segundo clube (aquele que fica geograficamente mais longe de você) pro qual você torce?_y'])

In [45]:
df2.to_excel('Output do Python do Censo do r-futebol de Dezembro de 2022.xlsx', index = False)